Demo Code: 
> Graph-Based Behavioral and Emotional Analysis System

In [21]:
import argparse
import time
import math
import json
import torch
import cv2
from pathlib import Path
from sklearn.metrics.pairwise import cosine_similarity
from deepface import DeepFace
from IPython.display import display, Image, HTML
import matplotlib.pyplot as plt
import matplotlib.animation as animation

In [22]:
from emotion import detect_emotion, init
from models.experimental import attempt_load
from utils.datasets import LoadStreams, LoadImages
from utils.general import check_img_size, check_requirements, non_max_suppression, scale_coords, set_logging
from utils.torch_utils import select_device

from modules.functions import *

In [23]:
video_path = 'data/samples/Bully1Final.mp4'  
save_path = 'results.json'  

In [24]:
from IPython.display import HTML

def display_video(video_path):
    # Create an HTML video tag to embed the video in the notebook
    video_tag = f'''
    <video width="640" height="480" controls>
        <source src="{video_path}" type="video/mp4">
        Your browser does not support the video tag.
    </video>
    '''
    # Display the video
    display(HTML(video_tag))

# Display the video in the notebook
display_video(video_path)

# Display Emotions and Face Detection

In [26]:
display_video('data/samples/faceDetectionWithEmotion.mp4'  )

# Extract the influenced_rate of each node

In [25]:

# Initialize necessary functions
def get_face_embedding(face_image):
    """Get a unique embedding for a given face image."""
    embedding = DeepFace.represent(face_image, model_name='VGG-Face', enforce_detection=False)
    return embedding[0]['embedding']  # Returns a 128D vector (for VGG-Face)

def get_face_match(new_face_embedding, existing_faces_embeddings, threshold=0.5):
    """Compare the new face embedding with existing embeddings to find the closest match."""
    for i, existing_embedding in enumerate(existing_faces_embeddings):
        similarity = cosine_similarity([new_face_embedding], [existing_embedding])
        if similarity > threshold:
            return i  # Return the index of the matched face
    return None  # No match found

def calculate_distance(bbox1, bbox2):
    """Calculate Euclidean distance between the centers of two bounding boxes."""
    x1_center = (bbox1[0] + bbox1[2]) / 2
    y1_center = (bbox1[1] + bbox1[3]) / 2
    x2_center = (bbox2[0] + bbox2[2]) / 2
    y2_center = (bbox2[1] + bbox2[3]) / 2
    return math.sqrt((x2_center - x1_center) ** 2 + (y2_center - y1_center) ** 2)

# Main function to process detections
def process_detections_with_distances(video_path, save_path, img_size=512, conf_thres=0.5, iou_thres=0.45):
    webcam = False

    # Set up device and model
    set_logging()
    device = select_device('')
    init(device)
    half = device.type != 'cpu'
    model = attempt_load("weights/yolov7-tiny.pt", map_location=device)
    stride = int(model.stride.max())
    img_size = check_img_size(img_size, s=stride)
    if half:
        model.half()

    dataset = LoadImages(video_path, img_size=img_size, stride=stride)

    if device.type != 'cpu':
        model(torch.zeros(1, 3, img_size, img_size).to(device).type_as(next(model.parameters())))

    all_results = []
    frame_number = 0
    existing_faces_embeddings = []  # Store embeddings of faces detected in previous frames

    # Process video frames
    for path, img, im0s, vid_cap in dataset:
        img = torch.from_numpy(img).to(device)
        img = img.half() if half else img.float()
        img /= 255.0
        if img.ndimension() == 3:
            img = img.unsqueeze(0)

        pred = model(img)[0]
        pred = non_max_suppression(pred, conf_thres, iou_thres)

        frame_detections = []
        for det in pred:
            if len(det):
                det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0s.shape).round()
                images = []
                for det_item in det:
                    xyxy, conf, cls = det_item[:4], det_item[4], int(det_item[5])
                    x1, y1, x2, y2 = map(int, xyxy)
                    images.append(im0s[y1:y2, x1:x2])

                if images:
                    emotions = detect_emotion(images)

                for i, det_item in enumerate(det):
                    xyxy, conf, cls = det_item[:4], det_item[4], int(det_item[5])
                    x1, y1, x2, y2 = map(int, xyxy)
                    emotion_label, emotion_confidence = emotions[i]

                    face_embedding = get_face_embedding(images[i])  # Get embedding of the face
                    matched_node_id = get_face_match(face_embedding, existing_faces_embeddings)

                    if matched_node_id is None:
                        # Assign a new node ID if no match found
                        node_id = len(existing_faces_embeddings)
                        existing_faces_embeddings.append(face_embedding)
                    else:
                        # Assign the existing node ID if a match is found
                        node_id = matched_node_id
                    emot,perc = extract_emotion_and_percentage(emotion_label)
                    detection_data = {
                        "node_id": node_id,
                        "bbox": [x1, y1, x2, y2],
                        "emotion": emot,
                        "percentage": float(perc),
                    }
                    frame_detections.append(detection_data)
        # Compute distances between all detected nodes
        for det in frame_detections:
            det["distances"] = [
                {"node_id": other["node_id"], "distance": calculate_distance(det["bbox"], other["bbox"])}
                for other in frame_detections if det["node_id"] != other["node_id"]
            ]       
        all_results.append({"frame": frame_number, "detections": frame_detections})
        all_results.append({"frame": frame_number, "detections": frame_detections})
        frame_number += 1

    # Save results to JSON file
    with open(save_path, "w") as f:
        json.dump(all_results, f, indent=4)


process_detections_with_distances(video_path, save_path)

YOLOv5  2024-12-20 torch 2.5.1+cpu CPU

Fusing layers... 
Model Summary: 224 layers, 6048051 parameters, 0 gradients, 13.2 GFLOPS


video 1/1 (1/364) e:\KU\Events\2024 BIOHACKATHON nepalensis\data\samples\Bully1Final.mp4: video 1/1 (2/364) e:\KU\Events\2024 BIOHACKATHON nepalensis\data\samples\Bully1Final.mp4: video 1/1 (3/364) e:\KU\Events\2024 BIOHACKATHON nepalensis\data\samples\Bully1Final.mp4: video 1/1 (4/364) e:\KU\Events\2024 BIOHACKATHON nepalensis\data\samples\Bully1Final.mp4: video 1/1 (5/364) e:\KU\Events\2024 BIOHACKATHON nepalensis\data\samples\Bully1Final.mp4: video 1/1 (6/364) e:\KU\Events\2024 BIOHACKATHON nepalensis\data\samples\Bully1Final.mp4: video 1/1 (7/364) e:\KU\Events\2024 BIOHACKATHON nepalensis\data\samples\Bully1Final.mp4: video 1/1 (8/364) e:\KU\Events\2024 BIOHACKATHON nepalensis\data\samples\Bully1Final.mp4: video 1/1 (9/364) e:\KU\Events\2024 BIOHACKATHON nepalensis\data\samples\Bully1Final.mp4: video 1/1 (10/364) e:\KU\Events\2024 BIOHACKATHON nepalensis\data\samples\Bully1Final.mp4: video 1/1 (11/364) e:\KU\Events\2024 BIOHACKATHON nepalensis\data\samples\Bully1Final.mp4: video 1/

# Display the nodes and emotions
Used the file "Attempts\A graph video with fixed size and node.py"

In [30]:
display_video('./data/output/GraphWithEmotions.mp4'  )

# Relative Strength Index

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, FFMpegWriter

# Load data from CSV (replace this with the actual file path)
df = pd.read_csv('node_data.csv', header=0)

# Function to calculate RSI for each node
def calculate_rsi(df, period=50):
    delta = df.diff()  # Calculate price change between consecutive periods
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()  # Average gain
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()  # Average loss
    
    rs = gain / loss  # Relative Strength
    rsi = 100 - (100 / (1 + rs))  # RSI formula
    return rsi

# Calculate RSI for each node
rsi = calculate_rsi(df)

# Initialize figure
fig, axes = plt.subplots(df.shape[1], 1, figsize=(10, 8), sharex=True)

# Function to update each frame of the animation
def update(frame):
    print(frame)
    for i, ax in enumerate(axes):
        ax.clear()
        ax.plot(df.index[:frame], rsi.iloc[:frame, i], label=f'RSI Node_{i}')
        ax.axhline(70, color='red', linestyle='--', label='Threshold (70)')
        ax.axhline(30, color='green', linestyle='--', label='Oversold (30)')
        ax.set_title(f'Relative Strength Index for Node_{i}')
        ax.legend(loc='upper left')
    plt.tight_layout()

# Create animation
frames = len(df)  # Number of frames equals the number of rows in the data
ani = FuncAnimation(fig, update, frames=frames, interval=(1000/30), repeat=False)  # 30 FPS

# Save animation as a video
output_path = "rsi_animation.mp4"
writer = FFMpegWriter(fps=30, metadata=dict(artist='Your Name'), bitrate=1800)
ani.save(output_path, writer=writer)
print(f"Video saved at {output_path}")
